In [ ]:
from stmpy import Driver, Machine
import ipywidgets as widgets
from IPython.display import display
import paho.mqtt.client as mqtt
import json

group = "Group 2"

task_description = {
    1 : "Task 1: Create a state machine.",
    2 : "Task 2: Create a use case",
    3 : "Task 3: Draw a deployment diagram",
    4 : "Task 4: Write down your Vision",
    5 : "Task 5: Write down your Problem and Background"
}
task_answers ={
    "group": group,
    1 : "",
    2 : "",
    3 : "",
    4 : "",
    5 : ""
}


#Est. mqtt connection, student as publisher
mqttBroker = "mqtt.eclipseprojects.io"
client = mqtt.Client("Student_group-2")
client.connect(mqttBroker)

class Student:
    def __init__(self):
        print("Init - Student")
        self.counter = 0

        # Initilizing the UI
        self.button_next_task = widgets.Button(description="Next task")
        self.button_next_task.on_click(self.next_task_pressed)
        self.button_prev_task = widgets.Button(description="Previous task")
        self.button_prev_task.on_click(self.prev_task_pressed)
        self.button_help = widgets.Button(description="Help")
        self.button_help.on_click(self.help_pressed)

        self.text_area = widgets.Textarea(
            value='',
            disabled=False
        )

        self.label = widgets.Label(value = "")
        self.label_time = widgets.Label(value = "")

        # Displaying the UI
        display(self.button_next_task)
        display(self.button_prev_task)
        display(self.button_help)
        display(self.label)
        display(self.label_time)
        display(self.text_area)


    # First part of the program, always triggered when the program starts
    def show_tasks(self):
        self.stm.send('start_task')


    # Changes the "UI" to display the correct task
    def task_func(self):
        # When a group is working with the final task, the button changes description and
        # a the button is given functionality for delivering the tasks
        if self.counter == len(task_description) - 1:
            self.button_next_task.description = "Submit tasks"
        
        else:
            self.button_next_task.description = "Next task"
        #publish progress
        client.publish("Task", "{} {}".format(group, self.counter + 1))
        # "UI" changes according to what task the group is working on
        self.label.value = "{}".format(task_description[self.counter + 1])
        self.label_time.value = "{} minutes".format(int(self.stm.get_timer("t")/(1000*60)%60))
        self.text_area.value = '{}'.format(task_answers[self.counter + 1])
        
    # This function is triggered each time the next task/ deliver button is pressed
    def next_task_pressed(self, b):
        # Updates a dictionary with a groups answers
        task_answers[self.counter + 1] = self.text_area.value
        # Makes sure a group do not deliver blank
        if self.text_area.value.strip() == "":
            print("You must submit answer to continue")
        else:
            # Different functionalities if a group is working with the final task
            if self.button_next_task.description == "Submit tasks":
                print("Delivered")
                self.stm.send('deliver')
                # Publish delivered
                client.publish("Deliver", "{} delivered".format(group))
                encode_task_answers = json.dumps(task_answers, indent=2).encode('utf-8')
                client.publish("Deliver_save", encode_task_answers)
            else:
                self.counter += 1
                self.stm.send('next_task')


    # Functionality for the previous task button
    def prev_task_pressed(self, b):
        # Handelig that a group cannot try to access a task previous Task 1 
        if self.counter != 0:
            task_answers[self.counter + 1] = self.text_area.value
            self.counter -= 1
            self.stm.send('prev_task')


    # Handelig the event for clicking Help or Help finished
    def help_pressed(self, b):
        if self.button_help.description == "Help":
            self.button_help.description = "Help finished"
            self.stm.send('help')
        else:
            self.button_help.description = "Help"
            self.stm.send('help_finished')
    
    def add_group_to_queue(self):
        print("Added to queue")
        # Publish that group needs help
        client.publish("Help", "{}".format(group))
        self.button_help.description = "Help finished"

    def remove_group_from_queue(self):
        print("Removed from queue")
        # Publish that group do not require help
        client.publish("Help_finished", "{}".format(group))
        self.button_help.description = "Help"

    # When a group have delivered, the stm stops
    def finished(self):
        self.stm.driver.stop()



#for student
s0 = {"source": "initial", "target": "display_tasks"}

s1 = {
    "trigger": "start_task",
    "source": "display_tasks",
    "target": "task"
}
s2 = {
    "trigger": "next_task",
    "source": "task",
    "target": "task"
}
s3 = {
    "trigger": "help",
    "source": "task",
    "target": "TA_assistance"
}
s4 = {
    "trigger": "t",
    "source": "task",
    "target": "TA_assistance"
}
s5 = {
    "trigger": "deliver",
    "source": "task",
    "target": "end"
}
s6 = {
    "trigger": "back_to_waiting",
    "source": "task",
    "target": "display_tasks"
}
s7 = {
    "trigger": "help_finished",
    "source": "TA_assistance",
    "target": "task"
}

s8 = {
    "trigger": "prev_task",
    "source" : "task",
    "target" : "task",
}


#student states
display_tasks = {'name': 'display_tasks',
      'entry': 'show_tasks'
      }
task = {'name': 'task',
      'entry': 'start_timer("t", 1200000); task_func'
      }
TA_assistance = {'name': 'TA_assistance',
      'entry': 'stop_timer("t"); add_group_to_queue',
      'exit': 'remove_group_from_queue'
      }
end = {'name': 'end',
      'entry': 'finished'
      }


student = Student()
student_machine = Machine(name='stm_student', transitions=[s0, s1, s2, s3, s4, s5, s6, s7, s8], obj=student, states=[display_tasks, task, TA_assistance, end])
student.stm = student_machine

driver = Driver()
driver.add_machine(student_machine)
driver.start()